In [ ]:
# Colab Gradio App: Big Data Maturity Survey (20 Questions) + Scoring
# Run this cell in Google Colab

!pip -q install gradio

import gradio as gr

# -----------------------------
# Helpers: scoring + labels
# -----------------------------
LIKERT_LABELS = {
    1: "1 (غير متحقق)",
    2: "2 (متحقق بدرجة منخفضة)",
    3: "3 (متحقق بدرجة متوسطة)",
    4: "4 (متحقق بدرجة عالية)",
    5: "5 (متحقق بالكامل)",
}

def dim_level(score_4_to_20: int) -> str:
    # Per your ranges:
    # 4–8 منخفض, 9–13 متوسط, 14–17 متقدم, 18–20 راسخ
    if 4 <= score_4_to_20 <= 8:
        return "نضج منخفض"
    if 9 <= score_4_to_20 <= 13:
        return "نضج متوسط"
    if 14 <= score_4_to_20 <= 17:
        return "نضج متقدم"
    if 18 <= score_4_to_20 <= 20:
        return "نضج راسخ"
    return "خارج النطاق"

def overall_level(total_20_to_100: int) -> tuple[str, str]:
    # Per your ranges:
    # 20-39 الوعي, 40-59 مبادرات تجريبية, 60-74 استخدام منهجي, 75-89 تكامل مؤسسي, 90-100 استدامة وابتكار
    if 20 <= total_20_to_100 <= 39:
        return ("مستوى الوعي", "المؤسسة تدرك مفهوم البيانات الضخمة لكنها لا تطبّقه بعد")
    if 40 <= total_20_to_100 <= 59:
        return ("مبادرات تجريبية", "المؤسسة بدأت مشاريع محدودة وغير مستدامة في مجال البيانات الضخمة")
    if 60 <= total_20_to_100 <= 74:
        return ("استخدام منهجي", "المؤسسة تستخدم البيانات الضخمة بشكل منظم لكنه غير مدمج بالكامل في العمليات")
    if 75 <= total_20_to_100 <= 89:
        return ("تكامل مؤسسي", "المؤسسة دمجت البيانات الضخمة داخل أعمالها التشغيلية وصناعة القرار")
    if 90 <= total_20_to_100 <= 100:
        return ("استدامة وابتكار", "المؤسسة تصل إلى مرحلة النضج المتقدم وتُحسّن وتبتكر باستخدام البيانات الضخمة باستمرار")
    return ("خارج النطاق", "تحقق من القيم المدخلة")

QUESTIONS = [
    # 1) Institutional/Organizational (1-4)
    "1) توجد رؤية مؤسسية واضحة تدعم استخدام البيانات الضخمة في تطوير السياسات والخدمات الحكومية.",
    "2) توجد سياسات أو لوائح داخلية تنظّم تبنّي البيانات الضخمة وتحدد مسؤوليات الجهات المعنية.",
    "3) يتم تنفيذ مبادرات رسمية أو برامج حكومية مرتبطة بالبيانات الضخمة وليس مجرد جهود فردية أو تجريبية.",
    "4) يوجد تنسيق فعّال بين الإدارات الحكومية المختلفة لتبادل البيانات واستخدامها بشكل تكاملي.",

    # 2) Technical Infrastructure (5-8)
    "5) تتوفر بنية تقنية قادرة على استيعاب وتخزين ومعالجة البيانات واسعة الحجم والمتعددة المصادر.",
    "6) تمتلك الجهة إمكانية الوصول إلى موارد حوسبة قابلة للتوسع (سحابية/هجينة/مركز بيانات متقدم).",
    "7) توجد آليات تكامل بين الأنظمة الحكومية المختلفة تسمح بتوحيد البيانات وتبادلها.",
    "8) توجد ضوابط أمنية وحلول حماية سيبرانية تدعم التعامل مع البيانات الضخمة عالية الحساسية.",

    # 3) Data Governance & Management (9-12)
    "9) توجد سياسات واضحة لإدارة جودة البيانات وضمان دقتها واتساقها واكتمالها.",
    "10) يتم تطبيق معايير للخصوصية وحماية البيانات تتوافق مع التشريعات الوطنية.",
    "11) توجد آليات لتوثيق البيانات وتصنيفها وتحديد مستويات الوصول إليها داخل الجهة الحكومية.",
    "12) توجد ممارسات أو اتفاقيات لمشاركة البيانات بين الجهات الحكومية بطريقة منظمة وآمنة.",

    # 4) Human Capabilities & Skills (13-16)
    "13) لدى الجهة موظفون يمتلكون مهارات تحليل البيانات الضخمة ومعالجة مصادرها المتنوعة.",
    "14) يوجد برنامج تدريبي أو خطة تطوير مهني لتعزيز مهارات الموظفين في تحليل البيانات المتقدمة.",
    "15) تمتلك الجهة القدرة على تشغيل مشاريع البيانات الضخمة داخلياً دون اعتماد كامل على مزودين خارجيين.",
    "16) توجد فرق عمل متعددة التخصصات تجمع بين التحليل والبيانات والتقنية وصنع القرار.",

    # 5) Analytics & Operational Use (17-20)
    "17) تستخدم الجهة البيانات الضخمة لتحسين الخدمات الحكومية أو رفع جودة القرارات.",
    "18) توجد مشاريع تحليلية تطبيقية تستخدم البيانات الضخمة ضمن نطاق تشغيلي وليس تجريبي فقط.",
    "19) يتم استخدام أدوات تحليل متقدمة مثل النماذج التنبؤية أو التحليلات السلوكية أو التحليل اللحظي.",
    "20) توجد مؤشرات تقيس أثر استخدام البيانات الضخمة على الأداء الحكومي أو رضا المستفيدين.",
]

DIM_NAMES = [
    "البعد المؤسسي والتنظيمي",
    "البنية التحتية التقنية",
    "حوكمة وإدارة البيانات",
    "القدرات البشرية والمهارات",
    "التطبيقات التحليلية والاستفادة التشغيلية",
]

def compute_results(org, tech, gov, people, apps, entity_name):
    # Flatten answers in order
    answers = org + tech + gov + people + apps

    # Dimension sums (each is 4 questions, each 1-5 => 4-20)
    dim_scores = [
        sum(org),
        sum(tech),
        sum(gov),
        sum(people),
        sum(apps),
    ]

    total = sum(dim_scores)

    # Dimension level texts
    dim_lines = []
    for name, score in zip(DIM_NAMES, dim_scores):
        dim_lines.append(f"- {name}: **{score}/20** ({dim_level(score)})")

    overall_class, overall_meaning = overall_level(total)

    header = f"### نتائج قياس نضج البيانات الضخمة\n\n**الجهة/الوحدة:** {entity_name or 'غير محددة'}\n\n"
    dims_md = "#### درجات الأبعاد\n" + "\n".join(dim_lines) + "\n\n"
    total_md = f"#### النتيجة الكلية\n**المجموع الكلي:** **{total}/100**\n\n**التصنيف النهائي:** **{overall_class}**\n\n**المدلول:** {overall_meaning}\n"

    # Discussion prompts (brief, neutral)
    discussion = (
        "\n\n#### أسئلة نقاش مختصرة\n"
        "- ما البعد الأعلى درجة؟ وما السبب الواقعي خلف ذلك؟\n"
        "- ما البعد الأقل درجة؟ وما العائق الأبرز (تقني/تنظيمي/حوكمي/مهاري)؟\n"
        "- ما إجراء واحد قابل للتنفيذ خلال 90 يومًا لرفع أقل بُعد؟\n"
    )

    return header + dims_md + total_md + discussion

def build_dim_inputs(prefix: str):
    # Returns a list of 4 sliders
    sliders = []
    for i in range(4):
        sliders.append(
            gr.Slider(
                minimum=1,
                maximum=5,
                step=1,
                value=3,
                label=f"{prefix}{i+1}",
            )
        )
    return sliders

# -----------------------------
# UI
# -----------------------------
with gr.Blocks(title="استبانة نضج البيانات الضخمة (حكومي)", theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        "# استبانة قياس نضج البيانات الضخمة في الجهات الحكومية\n"
        "أجب عن الأسئلة وفق الواقع الفعلي. كل سؤال من 1 إلى 5.\n\n"
        "**سلم التقييم:** 1 غير متحقق، 2 منخفض، 3 متوسط، 4 عالٍ، 5 متحقق بالكامل."
    )

    entity_name = gr.Textbox(label="اسم الجهة/الوحدة (اختياري)", placeholder="مثال: معهد الإدارة العامة - إدارة ...")

    # Five dimensions in tabs
    with gr.Tabs():
        with gr.TabItem("1) المؤسسي والتنظيمي"):
            q1 = gr.Slider(1, 5, step=1, value=3, label=QUESTIONS[0])
            q2 = gr.Slider(1, 5, step=1, value=3, label=QUESTIONS[1])
            q3 = gr.Slider(1, 5, step=1, value=3, label=QUESTIONS[2])
            q4 = gr.Slider(1, 5, step=1, value=3, label=QUESTIONS[3])

        with gr.TabItem("2) البنية التحتية التقنية"):
            q5 = gr.Slider(1, 5, step=1, value=3, label=QUESTIONS[4])
            q6 = gr.Slider(1, 5, step=1, value=3, label=QUESTIONS[5])
            q7 = gr.Slider(1, 5, step=1, value=3, label=QUESTIONS[6])
            q8 = gr.Slider(1, 5, step=1, value=3, label=QUESTIONS[7])

        with gr.TabItem("3) الحوكمة وإدارة البيانات"):
            q9  = gr.Slider(1, 5, step=1, value=3, label=QUESTIONS[8])
            q10 = gr.Slider(1, 5, step=1, value=3, label=QUESTIONS[9])
            q11 = gr.Slider(1, 5, step=1, value=3, label=QUESTIONS[10])
            q12 = gr.Slider(1, 5, step=1, value=3, label=QUESTIONS[11])

        with gr.TabItem("4) القدرات البشرية والمهارات"):
            q13 = gr.Slider(1, 5, step=1, value=3, label=QUESTIONS[12])
            q14 = gr.Slider(1, 5, step=1, value=3, label=QUESTIONS[13])
            q15 = gr.Slider(1, 5, step=1, value=3, label=QUESTIONS[14])
            q16 = gr.Slider(1, 5, step=1, value=3, label=QUESTIONS[15])

        with gr.TabItem("5) التطبيقات والتحليل"):
            q17 = gr.Slider(1, 5, step=1, value=3, label=QUESTIONS[16])
            q18 = gr.Slider(1, 5, step=1, value=3, label=QUESTIONS[17])
            q19 = gr.Slider(1, 5, step=1, value=3, label=QUESTIONS[18])
            q20 = gr.Slider(1, 5, step=1, value=3, label=QUESTIONS[19])

    with gr.Row():
        btn = gr.Button("احسب النتائج", variant="primary")
        reset = gr.Button("تصفير إلى (3)")

    out = gr.Markdown()

    # Compute action
    btn.click(
        fn=lambda en, a1,a2,a3,a4, b1,b2,b3,b4, c1,c2,c3,c4, d1,d2,d3,d4, e1,e2,e3,e4:
            compute_results(
                [a1,a2,a3,a4],
                [b1,b2,b3,b4],
                [c1,c2,c3,c4],
                [d1,d2,d3,d4],
                [e1,e2,e3,e4],
                en
            ),
        inputs=[entity_name,
                q1,q2,q3,q4,
                q5,q6,q7,q8,
                q9,q10,q11,q12,
                q13,q14,q15,q16,
                q17,q18,q19,q20],
        outputs=out
    )

    # Reset to neutral (3)
    reset.click(
        fn=lambda: (3,)*20,
        inputs=[],
        outputs=[q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,q11,q12,q13,q14,q15,q16,q17,q18,q19,q20]
    )

demo.launch(share=True)


/tmp/ipython-input-3225209095.py:141: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="استبانة نضج البيانات الضخمة (حكومي)", theme=gr.themes.Soft()) as demo:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://48ff69044a2068990f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
